**นายพชรพล เกตุแก้ว รหัสนักศึกษา 6610110190**
# Food101-5: Lightweight Food Image Classifier Using MobileNetV2
- โมเดลจำแนกอาหาร 5 ชนิด


---

## ขั้นตอนที่ 1 นำเข้าไลบารี่ต่างๆที่จำเป็น

In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

โดยส่วนที่สำคัญสำหรับขั้นตอนนี้คือ การนำเข้าไลบารี่ต่างๆจาก tensirflow.keras ได้แก่

- **layers, models** -> ใช้สร้างชั้น (layer) ต่าง ๆ ของโมเดลและสร้างโครงสร้างโมเดล
- **.preprocessing.image**
  - **ImageDataGenerator** -> ใช้สร้าง data augmentation และโหลดภาพจากโฟลเดอร์เป็น batch โดยอัตโนมัติ
- **.callbacks**
  - **EarlyStopping** -> ใช้ในการหยุดฝึกอัตโนมัติหาก validation loss ไม่ดีขึ้นหลังจากผ่าน epoch ที่กำหนด → ป้องกัน overfitting
  - **ReduceLROnPlateau** -> ลด learning rate เมื่อ validation loss หยุดลดลง → ช่วยให้โมเดลเรียนรู้ละเอียดขึ้นในช่วงท้าย

## ขั้นตอนที่ 2 ดาวน์โหลดชุดข้อมูลและการกรอง

In [ ]:
# ดาวน์โหลด Dataset
!wget -nc http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar -xzf food-101.tar.gz

# สร้างโฟลเดอร์ Subset
subset_dir = "food101_5class"
os.makedirs(f"{subset_dir}/train", exist_ok=True)
os.makedirs(f"{subset_dir}/valid", exist_ok=True)

TOP_5_CLASSES = ['apple_pie', 'pizza', 'hamburger', 'sushi', 'ice_cream']

# สร้างโฟลเดอร์ย่อยแต่ละ Class
for cls in TOP_5_CLASSES:
    os.makedirs(f"{subset_dir}/train/{cls}", exist_ok=True)
    os.makedirs(f"{subset_dir}/valid/{cls}", exist_ok=True)

# อ่านรายการ Train/Test
with open("food-101/meta/train.txt") as f:
    train_list = [line.strip() for line in f.readlines()]
with open("food-101/meta/test.txt") as f:
    test_list = [line.strip() for line in f.readlines()]

# กรองเฉพาะ 5 class
train_filtered = [item for item in train_list if item.split('/')[0] in TOP_5_CLASSES]
valid_filtered = [item for item in test_list if item.split('/')[0] in TOP_5_CLASSES]

# คัดลอกไฟล์
for item in train_filtered:
    src = f"food-101/images/{item}.jpg"
    dst = f"{subset_dir}/train/{item}.jpg"
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copy(src, dst)

for item in valid_filtered:
    src = f"food-101/images/{item}.jpg"
    dst = f"{subset_dir}/valid/{item}.jpg"
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copy(src, dst)

print("สร้าง subset 5 class เสร็จสิ้น!")

File ‘food-101.tar.gz’ already there; not retrieving.

สร้าง subset 5 class เสร็จสิ้น!


ส่วนของขั้นตอนนี้ทำหน้าที่
- ดาวน์โหลดข้อมูลจาก http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz และแตกไฟล์
- จากนั้นสร้างโฟลเดอร์ที่มีชุดฝึกกับชุดทดลอง จากนั้นอ่านไฟล์ meta เผื่อกรองเอาเฉพาะ 5 class ที่เราเลือกมา
- สุดท้ายทำการคัดลอกไฟล์ต่างๆที่เราได้กรองมาจากใน meta มาไว้ในชุดฝึกและชุดทดลอง
- และแสดงผล "สร้าง subset 5 class เสร็จสิ้น!" เมื่อสำเร็จ

In [ ]:
# กำหนดพารามิเตอร์พื้นฐานสำหรับการประมวลผลภาพ
BATCH_SIZE = 32

# ปรับสเกลค่าพิกเซล + data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# ปรับสเกลค่าพิกเซลเท่านั้น
val_datagen = ImageDataGenerator(rescale=1./255)

# สร้างข้อมูลสำหรับชุดฝึกจากโฟลเดอร์
train_gen = train_datagen.flow_from_directory(
    f"{subset_dir}/train",
    batch_size=BATCH_SIZE,
    target_size=(224, 224),
    class_mode='categorical'
)

# สร้างข้อมูลสำหรับชุดตรวจสอบจากโฟลเดอร์
val_gen = val_datagen.flow_from_directory(
    f"{subset_dir}/valid",
    batch_size=BATCH_SIZE,
    target_size=(224, 224),
    class_mode='categorical'
)

# แสดง mapping ของ class
print(train_gen.class_indices)

Found 3750 images belonging to 5 classes.
Found 1250 images belonging to 5 classes.
{'apple_pie': 0, 'hamburger': 1, 'ice_cream': 2, 'pizza': 3, 'sushi': 4}


ส่วนของขั้นตอนนี้ทำหน้าที่
- กำหนดขนาดแบตช์เป็น 32 และตั้งค่า ImageDataGenerator สำหรับชุดฝึกให้ปรับสเกลค่าพิกเซลและเพิ่มความหลากหลายของข้อมูลด้วยการแปลงภาพ (data augmentation) เช่น หมุน เลื่อน พลิก และซูม
- ตั้งค่า ImageDataGenerator สำหรับชุดตรวจสอบให้ปรับสเกลค่าพิกเซลเท่านั้น โดยไม่มีการแปลงภาพเพิ่ม
- สร้างตัวสร้างข้อมูล (generator) สำหรับชุดฝึกและชุดตรวจสอบโดยอ่านภาพตรงจากโฟลเดอร์ย่อยของแต่ละคลาสอัตโนมัติ
- แสดง mapping ระหว่างชื่อคลาสกับ Index ที่โมเดลจะใช้ในการทำนาย เพื่อให้รู้ลำดับของแต่ละหมวดอาหาร

## ขั้นตอนที่ 3 สร้างและฝึกโมเดลจำแนกอาหารด้วย Transfer Learning

In [ ]:
# โหลดโมเดล MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

# สร้างโครงสร้างโมเดลใหม่ด้วย Functional API โดยต่อทับบน base_model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(5, activation='softmax')(x)

# ประกอบโมเดลสมบูรณ์จาก input ถึง output
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# กำหนด optimizer, loss function และ metrics สำหรับการฝึก
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ตั้งค่า callbacks สำหรับควบคุมการฝึก: หยุดอัตโนมัติเมื่อไม่ดีขึ้น + ปรับ learning rate
callbacks = [
    EarlyStopping(patience=8, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=4, min_lr=1e-7)
]

# เริ่มฝึกโมเดลโดยใช้ train_gen และ val_gen ที่เตรียมไว้
history = model.fit(
    train_gen,
    epochs=40,
    validation_data=val_gen,
    callbacks=callbacks
)

Epoch 1/40
118/118 ━━━━━━━━━━━━━━━━━━━━ 95s 652ms/step - accuracy: 0.5617 - loss: 1.1482 - val_accuracy: 0.8728 - val_loss: 0.3784 - learning_rate: 0.0010
Epoch 2/40
118/118 ━━━━━━━━━━━━━━━━━━━━ 51s 428ms/step - accuracy: 0.8014 - loss: 0.5645 - val_accuracy: 0.8912 - val_loss: 0.3182 - learning_rate: 0.0010
Epoch 3/40
118/118 ━━━━━━━━━━━━━━━━━━━━ 50s 421ms/step - accuracy: 0.8042 - loss: 0.5301 - val_accuracy: 0.8880 - val_loss: 0.3057 - learning_rate: 0.0010
Epoch 4/40
118/118 ━━━━━━━━━━━━━━━━━━━━ 49s 416ms/step - accuracy: 0.8087 - loss: 0.5193 - val_accuracy: 0.8936 - val_loss: 0.2844 - learning_rate: 0.0010
Epoch 5/40
118/118 ━━━━━━━━━━━━━━━━━━━━ 49s 416ms/step - accuracy: 0.8295 - loss: 0.4746 - val_accuracy: 0.8968 - val_loss: 0.2936 - learning_rate: 0.0010
Epoch 6/40
118/118 ━━━━━━━━━━━━━━━━━━━━ 49s 416ms/step - accuracy: 0.8253 - loss: 0.4661 - val_accuracy: 0.8952 - val_loss: 0.3050 - learning_rate: 0.0010
Epoch 7/40
118/118 ━━━━━━━━━━━━━━━━━━━━ 50s 426ms/step - accuracy: 0.8

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- โหลดโมเดลพื้นฐาน MobileNetV2 ที่ผ่านการฝึกมาแล้วบน ImageNet โดยตั้งค่า `include_top=False` เพื่อตัดชั้นสุดท้ายออก และ freeze น้ำหนักทั้งหมดไม่ให้ปรับระหว่างฝึก (Transfer Learning)  
- สร้างโครงสร้างโมเดลใหม่ด้วย Functional API โดยต่อชั้นประมวลผลเพิ่มเติมหลัง MobileNetV2 ได้แก่ GlobalAveragePooling2D, Dropout (เพื่อลด overfitting), ชั้น Dense ขนาด 128 ยูนิต และชั้นเอาต์พุตขนาด 5 ยูนิตพร้อม activation แบบ softmax สำหรับจำแนก 5 ชนิดอาหาร  
- คอมไพล์โมเดลด้วย optimizer ‘adam’, loss function ‘categorical_crossentropy’ และวัดผลด้วยความแม่นยำ (accuracy)  
- กำหนด callbacks สองตัว ได้แก่ EarlyStopping เพื่อหยุดฝึกอัตโนมัติหาก validation loss ไม่ดีขึ้นภายใน 8 epochs และ ReduceLROnPlateau เพื่อลด learning rate เมื่อการเรียนรู้ชะลอตัว  
- เริ่มฝึกโมเดลเป็นเวลา 40 epochs โดยใช้ข้อมูลจากตัวสร้างชุดฝึก (`train_gen`) และตรวจสอบกับชุดตรวจสอบ (`val_gen`) แบบ real-time

## ขั้นตอนที่ 4 ประเมินโมเดล

In [ ]:
# ดึงข้อมูลประวัติการฝึก
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# สร้างกราฟเปรียบเทียบทั้ง Accuracy และ Loss
plt.figure(figsize=(12, 4))

# กราฟที่ 1: Accuracy
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# กราฟที่ 2: Loss
plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# --- ประเมินผลเชิงตัวเลข ---
print("🔍 สรุปผลการฝึกโมเดล:")
print(f"- ความแม่นยำสูงสุดของชุดฝึก:     {max(acc):.4f} ({max(acc)*100:.2f}%)")
print(f"- ความแม่นยำสูงสุดของชุดตรวจสอบ: {max(val_acc):.4f} ({max(val_acc)*100:.2f}%)")

# ประเมินโมเดลบนชุดตรวจสอบอีกครั้งเพื่อความแม่นยำสุดท้าย
final_loss, final_accuracy = model.evaluate(val_gen, verbose=0)
print(f"\n✅ ความแม่นยำสุดท้ายบนชุดตรวจสอบ: {final_accuracy:.4f} ({final_accuracy*100:.2f}%)")

# ตรวจสอบว่าผ่านเกณฑ์ >80% หรือไม่
if final_accuracy > 0.80:
    print("🎉 ผ่านเกณฑ์! โมเดลพร้อมสำหรับการ deploy")
else:
    print("⚠️ ยังไม่ผ่านเกณฑ์ 80% ควรปรับปรุงเพิ่มเติม")

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- ดึงข้อมูลประวัติความแม่นยำและค่า loss จากการฝึกโมเดล เพื่อนำมาวิเคราะห์ผล  
- สร้างกราฟเปรียบเทียบระหว่างชุดฝึกและชุดตรวจสอบทั้งในด้าน **ความแม่นยำ (Accuracy)** และ **ค่าความสูญเสีย (Loss)** แบบคู่ขนาน เพื่อช่วยตรวจจับปัญหาเช่น overfitting หรือ underfitting  
- คำนวณและแสดงค่าความแม่นยำสูงสุดที่โมเดลทำได้ทั้งในชุดฝึกและชุดตรวจสอบ  
- ประเมินประสิทธิภาพสุดท้ายของโมเดลด้วย `model.evaluate()` บนชุดตรวจสอบอีกครั้งเพื่อให้ได้ค่าความแม่นยำที่แม่นยำจริง  
- ตรวจสอบว่าโมเดลผ่านเกณฑ์เป้าหมายที่กำหนดไว้ (>80%) หรือไม่ เพื่อตัดสินใจว่าพร้อมสำหรับการ deploy หรือยัง

## ขั้นตอนที่ 5 บันทึกโมเดล

In [ ]:
model.save("food_classifier_5class.keras")
print("✅ บันทึกโมเดลเรียบร้อย!")

## ขั้นตอนที่ 6 ทดสอบโมเดล

In [ ]:
# นำเข้าไลบรารีที่จำเป็นสำหรับการโหลดโมเดล ประมวลผลภาพ และอัปโหลดไฟล์
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt

# กำหนดขนาดภาพให้ตรงกับขนาดที่ใช้ฝึกโมเดล
IMG_SIZE = 224

# โหลดโมเดลที่บันทึกไว้จากไฟล์
model = load_model("food_classifier_5class.h5")

# อัปโหลดภาพจากผู้ใช้ผ่าน Google Colab
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

# โหลดและประมวลผลภาพให้พร้อมสำหรับการทำนาย
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# ทำนายคลาสของภาพด้วยโมเดล
pred = model.predict(img_array)
class_idx = np.argmax(pred)
confidence = pred[0][class_idx]
class_names = list(train_gen.class_indices.keys())
predicted_label = class_names[class_idx]

# แสดงผลลัพธ์การทำนายทางคอนโซล
print(f"🔍 ทำนาย: {predicted_label}")
print(f"📊 ความมั่นใจ: {confidence:.2%}")

# แสดงภาพพร้อมป้ายชื่อผลการทำนาย
plt.imshow(img)
plt.title(f"Prediction: {predicted_label} ({confidence:.2%})")
plt.axis('off')
plt.show()

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- โหลดโมเดลที่บันทึกไว้จากไฟล์เพื่อนำมาใช้ทำนายภาพใหม่  
- อนุญาตให้ผู้ใช้อัปโหลดภาพผ่านอินเทอร์เฟซของ Google Colab  
- โหลดและประมวลผลภาพที่อัปโหลดให้มีขนาดตรงกับที่โมเดลถูกฝึกมา (224×224 พิกเซล) พร้อมปรับสเกลค่าพิกเซลให้อยู่ในช่วง 0–1  
- แปลงภาพให้อยู่ในรูปแบบ batch ที่โมเดลรับได้ แล้วทำการทำนายคลาสของภาพ  
- ดึงชื่อหมวดอาหารจาก mapping ที่ได้จากชุดฝึก และแสดงผลลัพธ์การทำนายพร้อมระดับความมั่นใจเป็นเปอร์เซ็นต์  
- แสดงภาพที่อัปโหลดพร้อมป้ายชื่อผลการทำนายบนกราฟิกเพื่อให้เห็นผลลัพธ์อย่างชัดเจน